In [122]:
import cirq

## Deutsch's Algorithm
---------------------

Deutsch's algorithm is one of the earliset and simplest demonstrations of an problem that is easily solvable by a quantum computer, yet hard to solve by in a classical way.

### The Problem

The Deutsch (or Deutsch-Jozsa) problem consists of a black box 'oracle' function that takes in two bits and maps them to an ouput bit. The problem is formulated as such: 

"Is the oracle balanced or constant?"

Basically this is asking whether the outputs of the function depend on the input (no->constant; yes->balanced).

To solve this classically the function needs to be called at least twice (if acting on two bit only, the problem can be extended to *n* bits, but the quantum solution still holds.)

ie: if f(0) = 0, I still need to evaluate f(1) to determine whether the function is constant or balanced.

### The quantum solution

In a quantum computig setting, this can be solved via a single call to the oracle function rather than multiple calls. Needing to check the constant hypothesis $f(0) = f(1)$ (which is equivalent to checking whether $f(0) \bigoplus f(1)$ is 0), we consider a quantum implementation of the function $f$ that maps $\big|x\big>$$\big|y\big>$ to $\big|x\big>$$\big|f(x)\bigoplus y\big>$. This is similar to a CNOT gate mapping $\big|x\big>$$\big|y\big>$ to $\big|x\big>$$\big|x\bigoplus y\big>$.

Starting with two qubits in the preprepared state $|0> |1>$ we apply a Hadamard Gate to each qubit yielding:

H$\big|01\big> \rightarrow \frac{1}{2}(\big|0\big> + \big|1\big>)(\big|0\big> - \big|1\big>)$

Our 'quantum function' applied to the qubits will give us:

$
\frac{1}{2}\big|0\big>(\big|f(0)\bigoplus 0\big> - \big|f(0)\bigoplus1\big>) + \frac{1}{2}\big|1\big>(\big|f(1)\bigoplus 0\big> - \big|f(1)\bigoplus1\big>)
$ <br/><br/>
$
=(-1)^{f(0)}\frac{1}{2}\big(\big|0\big> + (-1)^{f(0) \bigoplus f(1)}\big|1\big>\big)(\big|0\big> - \big|1\big>)\qquad-(1)
$

Ignoring the last bit and a global phase we have:

$
\frac{1}{2}\big(\big|0\big> + (-1)^{f(0) \bigoplus f(1)}\big|1\big>\big)\qquad-(2)
$

We finally apply another Hadamard gate to this state and we get 

$
\frac{1}{2}(\big|0\big> + \big|1\big> + (-1)^{f(0) \bigoplus f(1)}\big|0\big> - (-1)^{f(0) \bigoplus f(1)}\big|1\big>)
$

which we factorize as:

$
\frac{1}{2}((1 +  (-1)^{f(0) \bigoplus f(1)})\big|0\big> + (1 -  (-1)^{f(0) \bigoplus f(1)})\big|1\big>)\qquad-(3)
$

If $f(0) \bigoplus f(1) = 0$ (constant function), the only possible solution is that we measure a 0, whilst if $f(0) \bigoplus f(1) = 1$ (balanced function) we can only measure a 1.

### How does this translate to code?

First we need to prepare the initial state by generating two qubits and applying a Pauli X gate to qubit 1.

We can then apply the Hadamard gate to both qubits 0 and 1.

In [131]:
q0, q1 = cirq.LineQubit.range(2)

circuit = cirq.Circuit([cirq.X(q1), cirq.H(q1), cirq.H(q0)])

In [132]:
# Define a simulator to debug the state
s = cirq.Simulator()

print(s.simulate(circuit))

measurements: (no measurements)
output vector: 0.5|00⟩ - 0.5|01⟩ + 0.5|10⟩ - 0.5|11⟩


We now need to define a quantum function that returns two outputs for the 2 qubits. 

We can make this "oracle quantum function" random so to not be influenced by knowing the result.

In [133]:
def oracle_quantum_function(q0, q1, secret_seed):
    """
    The secret_seed variable gives us the output of the oracle function, the following code then uses it to construct the modified CNOT gate
    """
    if secret_seed[0]:
        yield [cirq.CNOT(q0, q1), cirq.X(q1)]

    if secret_seed[1]:
        yield cirq.CNOT(q0, q1)

We can finally combine the full circuit below, adding in the measurement on the first qubit to obtain a definite answer and classify our function.

In [134]:
import random

In [135]:
secret_seed = [random.randint(0, 1) for _ in range(2)]

# For debugging
print("f(x) =  {}, {}".format(secret_seed[0], secret_seed[1]))

oracle = oracle_quantum_function(q0, q1, secret_seed)


f(x) =  0, 0


In [136]:

# Add the result of the modified CNOT/oracle function to the circuit
circuit.append(oracle)

# Apply the Hadamard gate to qubit 0 and measure it out
circuit.append([cirq.H(q0), cirq.measure(q0, key="result")])

#Running the circuit and measuring it out
result = s.run(circuit)

In [137]:
def check_result(result):

    print('Result of f(0)⊕f(1):')
    print(result)
    print()
    print("The secret function is :", end=" ")
    if list(result.histogram(key="result").keys())[0]:
        print("Balanced!")
    else:
        print("Constant")


In [138]:
check_result(result)

Result of f(0)⊕f(1):
result=0

The secret function is : Constant
